<a href="https://colab.research.google.com/github/ManaviSh/transitfeed/blob/master/learn_gtfs1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import shutil
import os
import zipfile

req = requests.get('http://www.bkk.hu/gtfs/budapest_gtfs.zip', stream=True)
with open('budapest_gtfs.zip', 'wb') as file:
    shutil.copyfileobj(req.raw, file)
    
if not os.path.exists('./data'):
    os.mkdir('./data')
    
with zipfile.ZipFile('budapest_gtfs.zip', 'r') as file:
    file.extractall('./data')

os.listdir('./data')

['feed_info.txt',
 'routes.txt',
 'shapes.txt',
 'calendar_dates.txt',
 'stops.txt',
 'stop_times.txt',
 'trips.txt',
 'pathways.txt',
 'agency.txt']

In [0]:
import pandas as pd

trips = pd.read_csv('data/trips.txt', low_memory=False)
shapes = pd.read_csv('data/shapes.txt', low_memory=False)
trips.head()

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,boarding_door
0,5100,B4709410,B47094ASZKP-0021,Mexikói út,0,B47094ASZKP-0021_01,1110,2,2.0,NaN
1,5100,B47094100,B47094ASZKP-0021,Vörösmarty tér,1,B47094ASZKP-0021_02,1109,2,2.0,NaN
2,5100,B47094101,B47094ASZKP-0021,Mexikói út,0,B47094ASZKP-0021_02,1110,2,2.0,NaN
3,5100,B47094102,B47094ASZKP-0021,Vörösmarty tér,1,B47094ASZKP-0021_02,1109,2,2.0,NaN
4,5100,B47094103,B47094ASZKP-0021,Mexikói út,0,B47094ASZKP-0021_02,1110,2,2.0,NaN


In [0]:
num_trips_by_shape = trips.groupby('shape_id').aggregate({'route_id': 'count'}).reset_index()
num_trips_by_shape.rename({'route_id': 'trip_count'}, axis='columns', inplace=True)

shapes_with_nums = shapes.merge(num_trips_by_shape, on='shape_id')
shapes_with_nums.head()

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled,trip_count
0,0285,100001,47.519648,19.148326,0.0,6
1,0285,100002,47.519666,19.148313,2.0,6
2,0285,100003,47.519839,19.148638,31.0,6
3,0285,100004,47.520584,19.150246,178.0,6
4,0285,100005,47.520674,19.150432,195.0,6


In [0]:
!pip install holoviews


     |████████████████████████████████| 4.2MB 6.8MB/s 
     |████████████████████████████████| 1.3MB 41.2MB/s 
     |████████████████████████████████| 8.6MB 28.4MB/s 
     |████████████████████████████████| 501kB 35.2MB/s 
  Created wheel for bokeh: filename=bokeh-2.0.2-cp36-none-any.whl size=9072534 sha256=eb8708f727ec98ed2d50e2bca713efd762e2ca835fb3af64b8a3bc2c5e8d4ae4
  Stored in directory: /root/.cache/pip/wheels/a3/5e/7d/49405a043a4714a1b66bb7a16dfe16a1f0db9217baac7c4fe3
  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=427625 sha256=7fa3ec2ab132b7019b9fd9c4e5d426513edfa9bd6e27ab3aee5880e0454fc02b
  Stored in directory: /root/.cache/pip/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793f74d1165f4d0104
Successfully built bokeh tornado
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.2 which is incompatible.
  Found existing installation: tornado 4.5.3
    Uninstalling tornado-4.5.3:
      Succ

In [0]:
from datashader.utils import lnglat_to_meters
import numpy as np

shapes_with_nums['coord_x'], shapes_with_nums['coord_y'] = \
    lnglat_to_meters(shapes_with_nums['shape_pt_lon'], shapes_with_nums['shape_pt_lat'])

def split_df_by(df, byvar):
    df_parts = []
    for level, df_part in df.groupby(byvar):
        empty = pd.DataFrame([[level if colname == byvar else np.NaN for colname in df.columns]],
                             columns=df.columns)
        df_parts.append(df_part.append(empty))
    return pd.concat(df_parts) 

sep_shapes = split_df_by(shapes_with_nums, 'shape_id')

In [0]:
import bokeh.plotting as bp
from bokeh.models.tiles import WMTSTileSource
import datashader as ds
import datashader.transfer_functions as tf
from datashader.bokeh_ext import InteractiveImage
from datashader.utils import export_image
import colorcet as cc

bp.output_notebook()

x_range=(2.101e6, 2.155e6)
y_range=(5.994e6, 6.052e6)

if not os.path.exists('./img'):
    os.mkdir('./img')

In [0]:
p = bp.figure(tools='pan,wheel_zoom,reset',
              plot_width=int(600),
              plot_height=int(600),
              x_range=x_range,
              y_range=y_range)

p.axis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

url = "https://cartodb-basemaps-b.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png"
tile_renderer = p.add_tile(WMTSTileSource(url=url))
tile_renderer.alpha = 1

def image_callback(x_range, y_range, w, h, color_fn=tf.shade):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.line(sep_shapes, 'coord_x', 'coord_y', agg=ds.sum('trip_count'))
    image = tf.shade(agg, cmap=cc.fire, how='eq_hist')
    return image
import holoviews as hv

export_image(image_callback(x_range=x_range, y_range=y_range, w=2000, h=2000),
             filename="BUD_schedule_fire", background='black')
hv.datashade(p, image_callback)

ImportError: ignored